In [1]:
%load_ext autoreload
%autoreload 2

from read_excel import *
from compute_state import *
from collections import namedtuple
from custom_types import *

In [2]:
# Example usage:
file_path = 'private/movimientos.xlsx'  # Set the filename to 'ordenes.xlsx'
df = read_excel_to_dataframe(file_path)
print(df.loc[0])

Descripcion            Dividendo en efectivo / MO
Ticker                                         MO
Tipo de Instrumento                       Cedears
Concertacion                  2024-05-03 00:00:00
Cantidad                                      0.0
Precio                                       -1.0
Liquidacion                   2024-05-03 00:00:00
Moneda                          Dólares C.V. 7000
Importe                                      6.18
Name: 0, dtype: object


In [5]:
# Create a State tuple containing the Balance tuple
current_state = State(Fondos=tuple(), Cedears=tuple(), Bonos=tuple(), Corporativos=tuple())

# Iterate through all rows of the DataFrame
for index, row in df[::-1].iterrows():
    
    # Call the update_state function for each row
    next_state = update_state(row, current_state)
    
    # Update the current state to the next state
    current_state = next_state

current_state

2020-09-17 00:00:00, State: State(Fondos=(instrument_t(name='BCRFA', amount=1382.7319),), Cedears=(), Bonos=(), Corporativos=())
2020-09-17 00:00:00, State: State(Fondos=(instrument_t(name='BCRFA', amount=1382.7319),), Cedears=(), Bonos=(), Corporativos=())
2020-09-17 00:00:00, State: State(Fondos=(instrument_t(name='BCRFA', amount=1382.7319),), Cedears=(), Bonos=(), Corporativos=())
2020-10-05 00:00:00, State: State(Fondos=(instrument_t(name='BCRFA', amount=2404.4746),), Cedears=(), Bonos=(), Corporativos=())
2020-10-05 00:00:00, State: State(Fondos=(instrument_t(name='BCRFA', amount=2404.4746),), Cedears=(), Bonos=(), Corporativos=())
2020-10-05 00:00:00, State: State(Fondos=(instrument_t(name='BCRFA', amount=2404.4746),), Cedears=(), Bonos=(), Corporativos=())
2020-10-27 00:00:00, State: State(Fondos=(), Cedears=(), Bonos=(), Corporativos=())
2020-10-27 00:00:00, State: State(Fondos=(instrument_t(name='BRTA', amount=1450.8691),), Cedears=(), Bonos=(), Corporativos=())
2020-11-04 00:

Exception: ('No matching function found for operation type:', 'Movimiento Manual / Conversión CV 7.000 a CV 10.000 (dólar MEP)')